In [21]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

from pyspark.rdd import RDD

from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql.types import *

import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook

output_notebook()

In [2]:
conf = SparkConf().setMaster("local").setAppName("Playground")
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/12 15:08:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [23]:
sc = SparkContext.getOrCreate(conf=conf)

In [20]:
filepath = "../data/FL_insurance_sample.csv"

In [31]:
sc.parallelize([1,2,3,4,5]).collect()

def rdd_from_list(sc, n):
    return sc.parallelize(range(n))

In [32]:
print(rdd_from_list(sc, 10).collect())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [33]:
result_rdd = rdd_from_list(sc, 3)
assert isinstance(result_rdd, RDD)
assert result_rdd.collect() == [0, 1, 2]

In [34]:
def load_file_to_rdd(sc, path):
    return sc.textFile(path)

In [36]:
result_rdd = load_file_to_rdd(sc, filepath)

assert isinstance(result_rdd, RDD)

In [5]:
def create_dataframe(spark, rdd, schema):
    """Creates a dataframe from an RDD and a schema"""
    return spark.createDataFrame(rdd, schema)
    

In [6]:
rdd = spark.sparkContext.parallelize([('1', 'a'), ('2', 'b'), ('3', 'c'), ('4', 'd'), ('5', 'e'), ('6', 'f')])
schema = StructType([StructField('id', StringType(), True), StructField('letter', StringType(), True)])

result_df = create_dataframe(spark, rdd, schema)

In [7]:
assert result_df.schema == schema
assert result_df.rdd.collect() == rdd.collect()

In [9]:
genders_rdd = spark.sparkContext.parallelize([('1', 'M'), ('2', 'F'), ('3', 'F'), ('4', 'M'), ('5', 'F'), ('6', 'M')])
grades_rdd = spark.sparkContext.parallelize([('1', 1.0), ('2', 2.0), ('3', 3.0), ('4', 4.0), ('5', 5.0), ('6', 6.0)])

genders_schema = StructType([StructField('ID', StringType(), True), StructField('gender', StringType(), True)])
grades_schema = StructType([StructField('ID', StringType(), True), StructField('grade', FloatType(), True)])

genders_df = create_dataframe(spark, genders_rdd, genders_schema)
grades_df = create_dataframe(spark, grades_rdd, grades_schema)

genders_df.createOrReplaceTempView('genders')
grades_df.createOrReplaceTempView('grades')

In [15]:
genders_df.filter(genders_df['ID'] > 2)

DataFrame[ID: string, gender: string]

In [16]:
genders_df[genders_df['ID'] > 2].show()

+---+------+
| ID|gender|
+---+------+
|  3|     F|
|  4|     M|
|  5|     F|
|  6|     M|
+---+------+



In [17]:
spark.sql('SELECT * FROM genders WHERE ID > 2').show()

+---+------+
| ID|gender|
+---+------+
|  3|     F|
|  4|     M|
|  5|     F|
|  6|     M|
+---+------+



In [18]:
genders_df[genders_df['ID'] > 2].show()

+---+------+
| ID|gender|
+---+------+
|  3|     F|
|  4|     M|
|  5|     F|
|  6|     M|
+---+------+



In [40]:
sc.parallelize([[1, 3], [2, 9]]).map(lambda row: row[0]).collect()

[1, 2]

In [47]:
def op1(sc, matrix):
    matrix = sc.parallelize(matrix).map(lambda row: row[0]*2).collect()
    matrix = sc.parallelize(matrix).map(lambda row: row[1]-3).collect()
    return matrix

In [48]:
matrix = [[1,3], [2,5], [8,9]]
matrix_rdd = sc.parallelize(matrix)
result_rdd = op1(sc, matrix_rdd)

assert isinstance(result_rdd, RDD)
assert result_rdd.collect() == [[2, 0], [4, 2], [16, 6]]

TypeError: 'RDD' object is not iterable